In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 긴 시퀀스 다루기 
긴 시퀀스를 RNN으로 훈련하다보면 많은 타임 스텝에 걸쳐 실행해야하므로 펼친 RNN이 매우 깊은 네트워크가 된다. 이러다 보면 그레디언트 소실과 폭주 문제를 가질 가능성이 높아진다.

## 불안정안 그레디언트
좋은 가중치 초기화, 빠른 옵티마이저, 드롭아웃, 그래디언트 클리핑 등등으로 문제를 완화할 수 있다.<br>수렴하지 않는 활성화 함수(ex)ReLU)는 별 도움이 되지 않는데 그레디언트 폭주를 막지 못하기 때문이다. 그렇기 때문에 tanh 같은 수렴하는 활성화 함수를 사용하는 것이 좋다.

일반적인 완전연결층에서 배치 정규화는 그레디언트 문제를 해결하는데 큰 효과를 보이지만 RNN에서는 효율적으로 사용하기 어렵다. 순환 층 안이 아니라 순환 층 사이(수평이 아닌 수직 방향)에 적용했을 때 없는 것보다는 조금 나은 효과를 내기는 하지만 큰 효과는 없다. 케라스에서는 각 간단히 순환 층 이전에 BatchNormalization 층을 추가한다.

RNN에서는 배치 정규화보다는 층 정규화(layer normalization)이 잘 맞는다. 배치 정규화와 비슷하지만 배치 차원에 대해 정규화하는 대신 특성 차원에 대해 정규화한다. 장점으로는 샘플에 독립적으로 타임 스텝마자 동적으로 필요한 통계를 계산할 수 있다는 것이다. 일반적으로 입력과 은닉 상태의 선형 조합 직후에 사용된다.

다음은 메모리 셀 안에 층 정규화를 구현한 코드이다.

In [3]:
# 현재 타임 스텝의 inputs과 이전 타임 스텝의 은닉 states
class LNSimpleRNNCell(keras.layers.Layer):
  def __init__(self, units, activation='tanh',**kwargs):
    super().__init__(**kwargs)
    self.state_size = units
    self.output_size = units
    self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,activation=None)
    self.layer_norm = keras.layers.LayerNormalization()
    self.activation = keras.activations.get(activation)
  def call(self, inputs, states):
    outputs, new_states = self.simple_rnn_cell(inputs, states)
    norm_outputs = self.activation(self.layer_norm(outputs))
    return norm_outputs, [norm_outputs]

In [ ]:
# 이 사용자 정의 셀을 사용하려면 keras.layers.RNN 층을 만들어 셀의 객체를 전달하면 된다.
model = keras.models.Sequential([
                                 keras.layers.RNN(LNSimpleRNNCell(20),return_sequences=True,
                                                  input_shape=[None,1]),
                                 keras.layers.RNN(LNSimpleRNNCell(20),return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])

비슷하게 타임 스텝 사이에 드롭아웃을 적용하는 사용자 정의 셀을 만들 수 있다. 하지만 더 간단한 방법이 있는데 (keras.layers.RNN 제외한) 모든 순환 층과 케라스에서 제공하는 모든 셀은 "dropout"과 "recurrent_dropout" 매개변수를 지원한다. <br>
- dropout : 타임 스텝마다 입력에 적용하는 드롭아웃 비율
- recurrent_dropout : 타임 스텝마다 은닉 상태에 대한 드롭아웃 비율

그렇기 때문에 RNN에서 타임 스텝마다 드롭아웃을 적용하기 위해 굳이 사용자 정의 셀을 만들 필요가 없다.

# 단기 기억 문제 해결
일부 정보는 매 훈련 스텝 후 사라지게 된다. 많은 시간이 지나면 첫 입력의 흔적은 거의 사라지게 되는데 긴 문장의 번역시 이런 것들이 문제가 될 수 있다.

# LSTM (long short-term memory, 장단기 메모리) 셀
- h(t) : 단기 상태 벡터
- c(t) : 장기 상태 벡터
- x(t) : 입력 벡터

h(t-1)과 x(t) 두 입력으로 완전 연결층을 구성해 f(t),g(t),i(t),o(t) 네 벡터를 출력한다. g(t)는 tanh, 나머지는 sigmoid를 활성화 함수로 출력된 벡터들이다.<br>
1. c(t-1)이 0~1 사이의 값을 원소로 갖는 f(t)를 이용한 '삭제 게이트'를 통해 일부 기억을 잃는다.<br>(버릴 기억과 남길 기억을 구분하는 게이트)<br>
2. g(t)(주 층)은 현재 입력인 x(t)와 이전 단기 상태인 h(t-1)을 분석하는 일반적인 역할을 담당한다. i(t)가 제어하는 '입력 게이트'를 통해 g(t)의 어느 부분이 장기 기억에 더해져야하는지 제어한다. 이렇게 구성된 벡터가 삭제 게이트를 통과해 일부 기억을 잃어버린 상태의 장기 기억에 더해지게 된다.(=c(t))
3. o(t)가 제어하는 '출력 게이트'는 새로운 장기 기억(c(t))의 어느 부분을 읽어 h(t)와 y(t)로 출력해야하는지 제어한다.

요약하면 LSTM 셀은 중요한 입력을 인식하고 (입력 게이트), 장기 상태에 저장하고, 필요한 기간 동안 보존하고 (삭제 게이트), 필요할 때마다 이를 추출하기 위해 학습한다. <br>
시계열, 긴 텍스트, 오디오 녹음 등에서 장기 패턴을 잡아내는데 좋은 성과를 보인다.


In [ ]:
# 코드로의 구현은 매우 간단
model = keras.models.Sequential()
model.add(keras.layers.LSTM(20,return_sequences=True,input_shape=[None,1]))
model.add(keras.layers.LSTM(20,return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(10)))

# GRU (gated recurrent unit, 게이트 순환 유닛) 셀
LSTM의 간소화 버전이고 유사하게 동작한다.<br>
h(t-1)과 x(t)로 완전 연결층을 구성해 r(t)와 z(t)를 출력, h(t-1)에서 r(t)와의 연산을 거친 벡터와 x(t)를 완전 연결해 g(t)를 출력, g(t)는 tanh, r(t),z(t)는 sigmoid를 활성화 함수로 사용
1. 두 상태 벡터가 h(t) 하나로 합쳐졌다.
2. 하나의 게이트 제어기 z(t)가 삭제 게이트와 입력 게이트를 모두 제어한다. z(t)는 삭제 게이트, 1-z(t)는 입력 게이트.
3. 이전 상태(h(t-1))의 어느 부분이 주 층(g(t))에 노출될지는 r(t)로 제어
4. 이전 상태(h(t-1))에서 삭제 게이트를 거친 벡터와, g(t)를 입력 게이트에 통과시킨 후 나온 벡터를 더해(+) h(t)와 y(t)를 구성

keras.layers.GRU 층이 제공된다.

LSTM과 GRU는 단순 RNN보다 훨씬 긴 시퀀스를 다룰 수 있지만 매우 제한적인 단기 기억을 갖는다. 100 타임 스텝 이상의 시퀀스에서 장기 패턴을 학습하는데 어려움이 있기 때문에 1D 합성곱 층을 사용해 입력 시퀀스를 짧게 줄이는 방법을 쓴다.

# 1D 합성곱 층을 사용홰 시퀀스 처리
2D 합성곱 층과 마찬가지로 몇개의 커널 혹은 필터가 슬라이딩하여 각 커널마다 1D 특성 맵을 출력한다. 10개의 커널을 사용하면 10개의 1차원 시퀀스를 반환하는 것이다.<br>
stride=1,padding='same'을 사용하면 출력 시퀀스는 입력 시퀀스의 길이와 같아진다. padding='valid'를 사용하면 출력 시퀀스는 입력 시퀀스보다 짧아진다.

다음 예를 살펴보면 스트라이드를 2로 사용해 입력 시퀀스를 두 배로 다운샘플링하는 1D 합성곱 층으로 시작하게 한다. 커널 크기가 스트라이드보다 크므로 모든 입력을 사용해 이 층의 출력을 계산한다. 따라서 모델이 중요하지 않은 세부사항은 버리고 유용한 정보만을 보존하도록 학습이 가능해진다. <br>
그렇기 때문에 합성곱 층으로 시퀀스 길이를 줄이면 GRU 층이 더 긴 패턴을 감지하는데 도움이 된다,

In [ ]:
from tensorflow.keras.layers import Conv1D, GRU, TimeDistributed, Dense
model = keras.models.Sequential()
model.add(Conv1D(filters=20,kernel_size=4,strides=2,padding='valid',input_shape=[None,1]))
model.add(GRU(20,return_sequence=True))
model.add(GRU(20,return_sequence=True))
model.add(TimeDistributed(Dense(10)))

model.compile(loss='mse',optimizer='adam',metrics=[last_time_step_mse])
history = model.fit(~~)